In [1]:
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as ex

sys.path.insert(0, '..')
from scripts import exploratory_analysis
from scripts import dist
from matplotlib import pyplot as plt
from scipy import stats as ss

In [2]:
beneficiary_data = pd.read_pickle('../../data/cleaned/train_beneficiary.pkl')
inpatient_data = pd.read_pickle('../../data/cleaned/train_inpatient.pkl')
outpatient_data = pd.read_pickle('../../data/cleaned/train_outpatient.pkl')